1. Obtaining $t$ via Newton's Method (method to solve numerically a one variable equation) and Eq.(3) having substituted theta from Eq.(1)

In [1]:
#Defining variables
x = var('x')
k = var('k')
b = 0.07531865  #\gamma_{12,5} overall value (found through data) 
B = 0.61410382  #\Gamma_{12,5} oberall value (found through data)
a = 10^(-6)     #tolerance of error
n_iter = 10^6   #number of iterations in the numeric method


#Defining intermediate sums and the function f, of which we need to solve f(t)=0
sum1 = sum(binomial(12,k)*(x^(12-k)/(1+x)^12),k, 0, 5)
sum2 = sum(binomial(12,k)*(x^(k)/(1+x)^12), k, 0, 5)

f_expr = (b*(1+x)^11/(x^5*(x-1)*binomial(12,5)) - 1/(x^2-1)) *sum1 \
       + (-b*(1+x)^11/(x^5*(x-1)*binomial(12,5)) + x^2/(x^2-1)) *sum2 - B

f = fast_callable(f_expr, vars=[x], domain=RR)  #fast_callable function allows to evaluate \
                                                #numerically in a more efficiently than represetation 
fdiff = fast_callable(diff(f_expr, x), vars=[x], domain=RR)


#Defining general Newton's method function
def newton(f, fdiff, x0, a, n_iter):
    x = x0
    for i in srange(0, n_iter):
        x_new = x - f(x)/fdiff(x)
        if abs(x_new - x) <= a:
            return x_new
        else:
            x = x_new
    return x


#Finding root t
x0 = 1.5        #seed to start the numeric method
t = newton(f, fdiff, x0, a, n_iter)
t

2.91351963099215

In [2]:
#Finding root t with a new seed
x0 = 3
t = newton(f, fdiff, x0, a, n_iter)
t

2.91351963099238

In [3]:
#Finding root t with upgraded precision
a = 10^(-7)
n_iter = 10^7
x0 = 3
t = newton(f, fdiff, x0, a, n_iter)
t

2.91351963099215

In [4]:
#Checking any other possible roots near t=1 (remember our root satisfies t>1)
a = 10^(-7)
n_iter = 10^7
x0 = 1.001      #(x0 = 1) throws a division by 0
t = newton(f, fdiff, x0, a, n_iter)
t

2.91351963099215

In [5]:
#Checking any other possible roots positively further from t=2.913596...
a = 10^(-6)
n_iter = 10^6
for i in srange(1, 10):
    x0 = 10^i
    t = newton(f, fdiff, x0, a, n_iter)
    print(t)

2.91351963099215
2.91351963099225
2.91351963099215
2.91351963099215
2.91351963099215
2.91351963099215
2.91351963099215
2.91351963099215
2.91351963099215


In [6]:
#This means that in the range [1,10^9] we only find one root to our function.
#Note that t>10^9 would imply mu-->1^{-} (mu practically equal to 1), which \
#is not realistically possible.

2. Defining a specific Newton's method for our function $f$ and checking it with the found value of $t$

In [7]:
#Defining Newton's method function specifically for our task (to change b and B too)
reset()

def newton_t(b, B, x0, a, n_iter):
    x = var('x')
    k = var('k')
    sum1 = sum(binomial(12,k)*(x^(12-k)/(1+x)^12),k, 0, 5)
    sum2 = sum(binomial(12,k)*(x^(k)/(1+x)^12), k, 0, 5)
    f_expr = (b*(1+x)^11/(x^5*(x-1)*binomial(12,5)) - 1/(x^2-1)) *sum1 \
       + (-b*(1+x)^11/(x^5*(x-1)*binomial(12,5)) + x^2/(x^2-1)) *sum2 - B
    f = fast_callable(f_expr, vars=[x], domain=RR)
    fdiff = fast_callable(diff(f_expr, x), vars=[x], domain=RR)
    
    x = x0
    for i in srange(0, n_iter):
        x_new = x - f(x)/fdiff(x)
        if abs(x_new - x) <= a:
            return (i, x_new)  #to know the number of iterations needed
        else:
            x = x_new
    return x

In [8]:
#Checking the already found value with the specific method
b = 0.07531865
B = 0.61410382  
x0 = 3
a = 10^(-6)     
n_iter = 10^6
print(newton_t(b, B, x0, a, n_iter))

(2, 2.91351963099238)


In [9]:
#Upgrading tolerance since the number of iterations done is tiny
a = 10^(-15)     
n_iter = 10^6
print(newton_t(b, B, x0, a, n_iter))

(4, 2.91351963099215)


3. Finding $\theta$, $\mu$, $p_{12,5}$ and $P_{12,5}$ values from the obtained $t$

In [11]:
t = newton_t(b, B, x0, a, n_iter)[1]

mu = t/(1+t)
tht = (b*(1+t)^11/(t^5*(t-1)*binomial(12,5)) - 1/(t^2-1))

n = 12; i = 5    #jury size and number of votes for acquittal
m = n-2*i        #majority size
p = tht*t^m/(tht*t^m + 1-tht)

k = var('k')
U = sum(binomial(n,k)*mu^(n-k)*(1-mu)^k, k, 0, i) 
V = sum(binomial(n,k)*mu^k*(1-mu)^(n-k), k, 0, i)
P = tht*U/(tht*U + (1-tht)*V)

mu, tht, p, P

(0.744475537549180, 0.647187332212810, 0.939654294656448, 0.990742276539118)

4. Obtaining the $t$, $\mu$, $\theta$, $p_{12,5}$ and $P_{12,5}$ values for all the other cases (different $b$ and $B$ obtained from data)

In [12]:
#Defining a function that calculates t, mu, tht, p and P (to avoid repeating code)
def table5_data(b, B, x0, a, n_iter, n, i):
    t = newton_t(b, B, x0, a, n)[1]
    
    mu = t/(1+t)
    tht = (b*(1+t)^11/(t^5*(t-1)*binomial(12,5)) - 1/(t^2-1))
    
    m = n-2*i
    p = tht*t^m/(tht*t^m + 1-tht)
    
    k = var('k')
    U = sum(binomial(n,k)*mu^(n-k)*(1-mu)^k, k, 0, i) 
    V = sum(binomial(n,k)*mu^k*(1-mu)^(n-k), k, 0, i)
    P = tht*U/(tht*U + (1-tht)*V)
    
    return (t, mu, tht, p, P)

#Defining the variables that will have the same value throughout all cases
x0 = 3
a = 10^(-10)
n_iter = 10^6
n = 12
i = 5

In [13]:
#Overall
   #With Gamma_{12,4} estimation from 1831 data:
b = 0.07531865
B = 0.61410382
print(table5_data(b, B, x0, a, n_iter, n, i))

    #With Gamma_{12,4} estimation from number of trials with conviction of 7 to 5/number of total trials from 1826 to 1830:
b = 0.071080528
print(table5_data(b, B, x0, a, n_iter, n, i))

(2.91351963099215, 0.744475537549180, 0.647187332212810, 0.939654294656448, 0.990742276539118)
(2.99052945033501, 0.749406685893259, 0.644393483236283, 0.941880961283254, 0.991635780330658)


In [14]:
#For crimes against persons
b = 0.122188589
B = 0.485336195
table5_data(b, B, x0, a, n_iter, n, i)

(2.04930960175120,
 0.672056914317356,
 0.549613483551632,
 0.836732579560406,
 0.943206148117184)

In [15]:
#For crimes against property
b = 0.056119613
B = 0.659536879
table5_data(b, B, x0, a, n_iter, n, i)

(3.39656308704858,
 0.772549607454558,
 0.681069638119894,
 0.960992771393007,
 0.995982788180530)

In [16]:
#For the Seine Department
   #With Gamma_{12,4} estimation from 1831 data:
b = 0.098321539
B = 0.651158024
print(table5_data(b, B, x0, a, n_iter, n, i))
 
   #With Gamma_{12,4} estimation from number of trials with conviction of 7 to 5/number of total trials from 1826 to 1830:
b = 0.065474182
print(table5_data(b, B, x0, a, n_iter, n, i))

(2.62730630888538, 0.724313329274007, 0.702207089161669, 0.942119318348982, 0.988786535118392)
(3.16942293249200, 0.760158655960019, 0.678186969654007, 0.954892547977184, 0.994432037640037)


5. Checking the $t$, $\mu$, $\theta$, $p_{12,5}$ and $P_{12,5}$ values from the article ($b$ and $B$ taken from the article) 

In [17]:
#Overall
   #With Gamma_{12,4} estimation from 1831 data:
b = 0.070600173
B = 0.609385343
print(table5_data(b, B, x0, a, n_iter, n, i))
 
   #With Gamma_{12,4} estimation from number of trials with conviction of 7 to 5/number of total trials from 1826 to 1830:
b = 0.071080528
print(table5_data(b, B, x0, a, n_iter, n, i))

(2.99076463237663, 0.749421453751717, 0.639315024783906, 0.940668507571244, 0.991453476911543)
(2.98173209986214, 0.748853018003239, 0.639623636712973, 0.940405044143609, 0.991351193685748)


In [18]:
#For crimes against persons
b = 0.115065903
B = 0.478213508
table5_data(b, B, x0, a, n_iter, n, i)

(2.11199315983422,
 0.678662532775852,
 0.535319013023972,
 0.837095768153116,
 0.946384188986507)

In [19]:
#For crimes against property
b = 0.052157469
B = 0.655574735
table5_data(b, B, x0, a, n_iter, n, i)

(3.49107335059820,
 0.777336079388059,
 0.674911245085781,
 0.961980735560679,
 0.996358483699078)

In [20]:
#For the Seine Department
   #With Gamma_{12,4} estimation from 1831 data:
b = 0.098054726
B = 0.650891211
print(table5_data(b, B, x0, a, n_iter, n, i))
 
   #With Gamma_{12,4} estimation from number of trials with conviction of 7 to 5/number of total trials from 1826 to 1830:
b = 0.065474182
print(table5_data(b, B, x0, a, n_iter, n, i))

(2.63042494830770, 0.724550152051444, 0.701709316186402, 0.942118955542153, 0.988815757373725)
(3.16893561424573, 0.760130620251633, 0.677918728151258, 0.954826327874835, 0.994421395442216)
